**Importing Libraries**

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import string


**Fuction for cleaning the text**

In [3]:
def clean_text(text):
    # Remove HTML tags
    clean_text = BeautifulSoup(text, "html.parser").get_text(separator=" ")
    # Remove extra whitespaces
    clean_text = " ".join(clean_text.split())
    # Remove punctuation
    clean_text = clean_text.translate(str.maketrans('', '', string.punctuation))
    return clean_text

**URL to extract data**

In [16]:
URL = "https://www.dawn.com/latest-news"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')
news = []
table = soup.find('div', attrs = {'class':'tabs__pane active'})
for row in table.findAll('div', attrs = {'class':'flex flex-col sm:flex-row'}):
    new = {}
    new['headline'] = clean_text(row.h2.text)
    new['description'] = clean_text(row.find('div', attrs = {'class':'story__excerpt font-georgia font-noto text-3.5 text-gray-700 overflow-hidden pb-1 px-0 sm:px-2 mt-0'}).text)
    new['url'] = row.a['href']
    try:
        new['img'] = row.img['src']
    except:
        continue
    new['Published Date'] = clean_text(row.find('span', attrs = {'class':'timestamp__calendar'}).text)
    news.append(new)


**Data saving in csv file**

In [18]:
# Write the cleaned data into a CSV file
filename = 'latestnews_dawn.csv'
with open(filename, 'w', newline='') as f:
    w = csv.DictWriter(f,['headline','description','url','img','Published Date'])
    w.writeheader()
    for n in news:
        w.writerow(n)